## TRAIN SET

In [ ]:
num_questions = [len(dataset['test'][i]['qas']['question']) for i in range(len(dataset['test']))]
print('Number of questions in test set: ', sum(num_questions)) 
plt.hist(num_questions, bins=12, range=(0.5, 12.5))
plt.title('Number of questions per article')
plt.xlabel('Number of questions')
plt.ylabel('Number of article')
plt.show()


In [ ]:
a = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
random.shuffle(a)
print(a)

In [ ]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)

flattened_dataset = datasets.DatasetDict({'train': train_dataset, 'test': test_dataset})

assert len(train_abstracts) == len(train_questions) == len(train_answers)
assert len(test_abstracts) == len(test_questions) == len(test_answers)

display(flattened_dataset['train'][0])

In [ ]:
# randomly choose 10% of the training data as validation data from flattened dataset
temp = flattened_dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=SEED)
flattened_dataset["train"] = temp["train"]
flattened_dataset["val"] = temp["test"]

display(flattened_dataset)

In [ ]:
encoded_ds = flattened_dataset.map(
    preprocess_function,
    batched=True,
    fn_kwargs={"max_qc_len": 128, "max_ans_len": 32},
)

In [ ]:
# load google/t5-efficient-mini
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-mini")

In [ ]:
# start wandb run
wandb.init()

trainer.train()

trainer.evaluate()

# end wandb run
wandb.finish()

In [ ]:
# run inference on a sample from the test split of flattened dataset
sample = flattened_dataset['test'][9]

# preprocess the sample
preprocessed_sample = preprocess_function(sample, max_qc_len=128, max_ans_len=32)

# move the sample to the device (GPU, if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# run inference
# model_base = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-mini")
# model_base.to(device)
generated_answer_ids = model.generate(
    input_ids=torch.tensor([preprocessed_sample['input_ids']]).to(device),
    attention_mask=torch.tensor([preprocessed_sample['attention_mask']]).to(device),
)

# decode the generated answer ids
generated_answer = tokenizer.decode(generated_answer_ids[0], skip_special_tokens=True)

print("Question:", sample['question'])
print("Context:", sample['abstract'])
print("Generated Answer:", generated_answer)
print("Actual Answer:", sample['answer'])